In [1]:
import tensorflow


In [2]:
print(tensorflow.__version__)

2.0.0


In [3]:
#我们将要预测 20 世纪 70 年代中期波士顿郊区房屋价格的中位数，已知当时郊区的一些数 据点，比如犯罪率、当地房产税率等。
#这里数据集所包含的数据点相对较少，只有 506 个，分为 404 个训练样本和 102 个测试样本。
#输入数据的 每个特征(比如犯罪率)都有不同的取值范围。
#例如，有些特性是比例，取值范围为 0~1;有 的取值范围为 1~12;还有的取值范围为 0~100，等等。

In [4]:
from tensorflow.keras.datasets import boston_housing

In [5]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [6]:
#看一下数据集大小

In [7]:
train_data.shape

(404, 13)

In [8]:
 test_data.shape

(102, 13)

In [9]:
train_data[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [10]:
#dataset downloading and introduction:https://www.kaggle.com/altavish/boston-housing-dataset

In [11]:
#数据特征的英语介绍
#CRIM - per capita crime rate by town
#ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
#INDUS - proportion of non-retail business acres per town.
#CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
#NOX - nitric oxides concentration (parts per 10 million)
#RM - average number of rooms per dwelling
#AGE - proportion of owner-occupied units built prior to 1940
#DIS - weighted distances to five Boston employment centres
#RAD - index of accessibility to radial highways
#TAX - full-value property-tax rate per $10,000
#PTRATIO - pupil-teacher ratio by town
#B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
#LSTAT - % lower status of the population
#MEDV - Median value of owner-occupied homes in $1000's

In [12]:
#目标是预测房价中位数，单位为千美元

In [13]:
#numpy实现标准化，因为各个数据的量纲不一样，比如面积、比入税率

In [14]:
mean = train_data.mean(axis=0) 
train_data -= mean
std = train_data.std(axis=0) 
train_data /= std
test_data -= mean 
test_data /= std

In [15]:
#由于样本数量很少，我们将使用一个非常小的网络，其中包含两个隐藏层，每层有 64 个单元。一般来说，训练数据越少，过拟合会越严重，而较小的网络可以降低过拟合。

In [16]:
#网络的最后一层只有一个单元，没有激活，是一个线性层。
#这是标量回归(标量回归是预 测单一连续值的回归)的典型设置。添加激活函数将会限制输出范围。
#例如，如果向最后一层 添加 sigmoid 激活函数，网络只能学会预测 0~1 范围内的值。
#这里最后一层是纯线性的，所以 网络可以学会预测任意范围内的值。

In [17]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [18]:
#函数封装模型

In [19]:
def build_model():
    model = models.Sequential() 
    model.add(layers.Dense(64, activation='relu',
    input_shape=(train_data.shape[1],))) 
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae']) 
    return model

In [20]:
#MSE， 均方误差，作为损失函数，同时会监控平均绝对误差

#做K折验证

In [21]:
import numpy as np
k=4
num_val_samples = len(train_data) // k 
num_epochs = 100
all_scores = []

为了在调节网络参数(比如训练的轮数)的同时对网络进行评估，你可以将数据划分为训 练集和验证集，正如前面例子中所做的那样。但由于数据点很少，验证集会非常小(比如大约 100 个样本)。因此，验证分数可能会有很大波动，这取决于你所选择的验证集和训练集。也就 是说，验证集的划分方式可能会造成验证分数上有很大的方差，这样就无法对模型进行可靠的 评估。
在这种情况下，最佳做法是使用 K 折交叉验证。这种方法将可用数据划分为 K 个分区(K 通常取 4 或 5)，实例化 K 个相同的模型，将每个模型在 K-1 个分区上训练，并在剩 下的一个分区上进行评估。模型的验证分数等于 K 个验证分数的平均值。这种方法的代码实现 很简单。

In [22]:
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0)
    # Evaluate the model on the validation data
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [23]:
#得到结果

In [24]:
all_scores

[2.4735055, 2.3732738, 2.4464724, 2.3988996]

In [25]:
#取均值

In [26]:
np.mean(all_scores)

2.423038

我们让训练时间更长一点，达到 200 个轮次。为了记录模型在每轮的表现，我们需要修改 训练循环，以保存每轮的验证分数记录。

In [27]:
num_epochs = 100
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['mae']
    all_mae_histories.append(mae_history)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [28]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

画出MAE和Epoch的曲线

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()


In [ ]:
训练到80个epoch就可以了，不然会过拟合

In [ ]:
# Get a fresh, compiled model.
model = build_model()
# Train it on the entirety of the data.
model.fit(train_data, train_targets,
          epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mae_score

In [ ]:
参考Francis Chollet《Python 深度学习》3.6的实验，请大家务必阅读此书